<a href="https://colab.research.google.com/github/lorenzosintini/4YP/blob/master/Progress_sheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **4YP PROGRESS AND PLANNING**

# Friday 18th October Meeting

## **Things to do next:**

### 1. Look at the latent space distribution when training the VAE with different classes.
-  a) Do different classes lie in different regions? 
-  b) Are similar number closer to each other (e.g. 1 and 7, 3 and 8…)? 
-  c) If we “zoom in” into a single class, is it possible to notice further classification or differences between different ways of writing the same number?

### 2. Train the VAE only with certain classes 
- a) How well does it reconstruct that same class and other classes (which would be novel)? 
- b) If it still reconstruct untrained pictures well (e.g. pictures of 1s seem to be easily reconstructible due to their simple shape), where would that picture lie in the latent space after being decoded? 
- c) Is it an outlier or far off from “typical” distribution of other pictures?

### 3. Compare a single VAE trained with all the classes vs multiple VAEs trained separately for each class
- a) Train 10 VAEs for each number, then input a test picture to each of them and find their loss when being reconstructed. The one with the smallest loss is the class it belongs to.
- b) Train 1 CVAE (conditional variational auto encoder) with all the numbers. Given a test picture find the reconstruction loss for each of the conditional cases (the 10 possible classes). The one with the smallest loss is the class it belongs to.
- c) Train 1 VAE with all the numbers. Rather than reconstructing a test image, determine which class it belongs to by looking at the latent vector constructed by the decoder (ideally each class should lie in a different latent space region)

### 4. Train the VAE (or CVAE) with different amounts of data for different classes. 
- a) How much worse is the reconstruction?
- b) Does this affect the categorisation performance (less train data for one category = larger test reconstruction error for that category—> might not think that it’s the category it belongs to, since better trained categories give smaller errors)

### 5.Make the code less dependent of the specific dataset used for. 
- a) Try to make the same code run for for MNIST-numbers and MNIST-fashion.

### 6.Try and get remote machine access in ORI 

### 7.Share the GitHub repository

### **Next Meeting:**

Friday 8th November

## **1**

**a)** 
Yes, the classes occupy different regions in the latent space. The separation between classes is more evident in some latent-space parameter than others: sometimes there is a lot of overlap while other times they are very clearly separated in different regions. For example if we train the model with classes 0,2 and 3 (which look quite different) the encoder maps training pictures in a latent space where, by plotting the 4th and 5th parameter in a scatter plot, we can notice different classes. 

*Could create a classifier in the latent space that determines the contours of these regions, when a test picture is tested we can tell which class it belongs to based on where it lies in the space.*


**b)**
Not necessarily, different parameters of the latent space might represent different features so classes might be close in certain parameters (if they both have the feature) but far in others (if they don’t have it). Tried training the VAE with 0,1 and 7, thinking that 7 and 1 were going to be close, but in reality the 7s were further apart than the 1s and 0s quite often. When trained on 2,5,7 it was a bit more noticeable that on average the classes of 2s and 7s were closer to each other compared to the 5s.

*Would be interesting to analyse or visualise what features the different parameters in the latent space represent.* 

Something weird noticed: when trained on the 1s, quite a few outliers emerged from that class. When plotting the latent space, all parameters are grouped close-is to each other with values in the range of [-500 to +500]. This time however (and only in certain parameter of the latent space) some images were encoded with some parameters of size 1e15 and up to 1e32. When trying to figure out why those particular z-values were so high, i noticed that it was due to their log-variance being positive and hence generating huge numbers. This only happened with pictures of 1s, if I get rid of the outliers everything looks normal like before. There were 371 outliers in the first 2000 pictures (of which about 6700 must have been 1s). Not sure why the variance would be positive in those cases when it’s always positive everywhere else.
Update: Tried training the VAE again and this did not happen anymore.


**c)**
Might not be able to classify different specific “ways” of writing the same number because the numbers are written in a spectrum of different ways and therefore distributed everywhere in the latent space as opposed to in defined region. However we can notice how numbers written similarly have similar values in some latent space parameters. The mapping of numbers (with specific features) in the latent space parameter can give us information as to what that parameter is trying to “look for”. For example the 30th and 22nd fours (both written wide, think and without sharp edges) have low values in the 4th latent space parameters. This suggests that that parameter might respond well to those features. Similarly the 16th four is written in a “closed” way (with the stick on the left tilted and forming a triangle with the vertical stick) and has low values in the 1st parameter.Unsurprisingly, the 47th and 166th fours (very low values in that parameter), also have that feature.

*If we can learn where certain uncommon features of a class lie in the latent space we could recognise them in the test data.*


## **2**

**a)**
When the VAE is only trained on certain classes, it will reconstruct test images with different loss results. The VAE was first trained with numbers 0s, 2, and 4s. The ELBO loss when reconstructing these classes was less than 100 and reconstruction was good (judging by the eye). The reconstruction of most of the other classes was quite bad and the images were distorted or looked like other numbers, their respective ELBO losses were around 200. However a few classes were better reconstructed. The 1s has a loss around 70 (better than many other trained classes). This is probably due to their simple shape, which the VAE has learnt to reconstruct (encoder needs to detect straight lines when trained on the 4s). However to the naked eye, the reconstruction seems worse than the one obtained for other classes (e.g. the number 2), even if the latter have higher ELBO losses. This might suggest that the ELBO loss might not be the best way to judge the reconstruction performance of the VAE. Maybe a mean square error between the image pixels would be better and worth trying.
A similar result was seen if the VAE is trained with 0,1,2,3,4. The ELBO losses of these classes were always lower than the ones for the other classes (class 9 was reconstructed better than the other untrained classes, only slightly worse than the trained classes). When images of 1s were provided into the training data, the 1s were reconstructed with much lower loss (only around 40).


**b)**
No. Even if a class is reconstructed well (without being trained with), there is not a very clear difference in the latent space position. The encoder maps pictures of 1s in a region similar/close to the one where the other values are located. In a few parameters of the latent vector it can be seen that the test images of 1s are grouped more closely together than other classes (or than they would in other parameters), but this is likely due to the fact that that parameter responds well to straight lines (as explained in 1)c)).


**c)**
No


## **3**

**a)**
10 Convolutional Variational Autoencoders were trained for each different class number. Since they were only trained for a single class, the latent space should look very similar (normal gaussian distribution). At testing time, each picture was tested on each VAE, and the VAE giving the smallest loss was chosen as the correct class for the test image. With a latent space with 2 dimensions, 89.8% accuracy was obtained while with a 10 dimensioanl latent space 98.2% accuracy was achieved. Further increasing the dimensionality of the latent space and tuning the hyperparameters coudld increase accuracy even more but I will not spend time on that now. Interestingly, it was noticed that sometimes (especially with the number 1 which is easy to learn/reconstruct) a model trained for a certain class (e.g. 2) gave lower reconstruction errors for novel classes (e.g. 1) rather than their own. This was not a problem for classification, since the model trained with class 1 gave even lower reconstruction errors and hence that model/class was picked as the correct one. However this would be a problem for novelty detection. If we weren't aware of the existance of class 1 and a novel picture from it was seen, the low reconstruction error from model_2 would lead us to belive that the test image belongs to class 2.

*Maybe using a different loss function for reconstruction (rather than the ELBO) will give better results. Need to investigate this more*

*Try to get a confidence level on the the classification: instead of binary make it a percentage, this is more sueful and could give us more insight. How to do this?*

**b)**
This time a single model was used. However it was a conditional variational autoencoder. This means that the labels of the pictures were fed to the encoder and decoder during the training. This means that we have now a new input (other than just the image pixels). The result is that all the pictures from each class are mapped into the same region of the latent space (normal gaussian) and are not separated. When we pick from it, is is the new class input to the decoder that will determine into which image that latent vector will be turned in. This allows us to pick a certain class without having to learn the latent space districution (much easier and convenient). For the classification, each test image is reconstructed 10 times, each time assuming it belongs to a different class (using that class vector as the extra input). The vector that lead to the smallest reconstruction error is considered as the 'right' assumption and the image in classified as that class. The accuracies achieved with a latent space of 2 and 10 dimensions were: 92.9% and 96.8%. The reason why this is lower could be due to the fact that the layers did not have any convolutional layers (which makes the network better). Hence the comparison is not very meaningful.

*Try and compare this to 10 VAEs rather than 10 Convolutional VAEs to compare which one is best. Alternitavely (and better) try to make a Convolutuional CVAE and compare it against the 3)a) model.*

*As before, try and get a confidence interval as percentage. This might be equivalent to finding the label vector that minimises the reconstruction (rather than trying 10 different ones and choosing the best). How to do this?*

**c)**
This idea makes less sense because it does not use the reconstructive ability of the VAE. If we classified images this way, then a normal classifier would be better and there is no need to distribute the images into a normal distribution or to even have a decoder in the first place.

## **4**

**a)**

The table shows the recontruction loss for each class when pictures form them are tested with the different class vectors. The diagonal entries are basically the correct reconstruction loss for each class.
To check how reconstruction is affected by the amount of data oer class, I considered 3 class distribution cases: 

**i)** Equal amount of training pictures for each class (5400 for class 0 to 9);
![alt text](https://drive.google.com/uc?id=1NPpVzdO4zGDjMGgYFc5x9n_yKpcXiSQj)

**ii)** One class with a lot more training data than the rest of them (5400 for class 5, 900 for all others);
![alt text](https://drive.google.com/uc?id=1Tl5sAciPvZUJyXecqpoPNyPtWbXv6ufM)

 
**iii)** Increasing distribution of traing pictures per class (200, 700, 1200, 1700, 2200, 2700, 3200, 3700, 4200, 4700, 5200, 5700 for classes 0 to 9)
![alt text](https://drive.google.com/uc?id=13BAIAHWqMeloHKuHEPGmttzPzwfYc1NJ)



**b)**
Similarly, the tables show, for each class, how many classifications counts were obtained from each class. Ideally, for each class, all pictures would be calssified as the correct value and the table would have zeros everywhere except the diagonal entries. For clarity, the same number of test images for each class were used (850 for all).

The same 3 classes distribution as before were used:

**i)**
![alt text](https://drive.google.com/uc?id=1nXuF-vx2hPyKHMiExcLUTkplrN-OaZt9)

**ii)**
![alt text](https://drive.google.com/uc?id=1s7c1X4nHtGLUthjAQvpI8zg5F-wIZrWn)

**iii)**
![alt text](https://drive.google.com/uc?id=1-fw5TQi6Bw6SLGkI557iBCE3979t1sp8)

## **7**

Done